In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from PIL import Image
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors

In [2]:
train = Path.cwd()/'training/images'

In [3]:
labels = pd.read_csv(Path.cwd()/'training/labels.csv')
labels['filename'] = labels.filename.apply(lambda x: int(x[2:6]))
labels.set_index('filename',inplace=True)
labels

,xRot
filename,
0,336.838887
1,148.484378
2,244.821661
3,222.700630
4,172.358121
...,...
4995,90.167663
4996,29.196271
4997,99.502175


In [4]:
images=[]
#each image is 512*512 ,lets chop it to 64*64
s=64
for img_path in tqdm(sorted(train.iterdir())):
    img = np.asarray(Image.open(img_path))[256-int(s/2):256+int(s/2),256-int(s/2):256+int(s/2),:]
    images.append(img.flatten().reshape(1,-1))
images = np.vstack(images)
images.shape

100%|██████████| 5000/5000 [00:19<00:00, 257.83it/s]


(5000, 12288)

In [5]:
img_names =[]
test_imgs = []
for img_path in tqdm((Path.cwd()/'test/images').iterdir()):
    img_name = img_path.name
    test_imgs.append( np.asarray(Image.open(img_path))
                     [256-int(s/2):256+int(s/2),256-int(s/2):256+int(s/2),:]
                     .flatten() )
    img_names.append(img_name)
test_imgs = np.vstack(test_imgs)
test_imgs.shape

5001it [00:19, 262.66it/s]


(5001, 12288)

In [6]:
nbrs = NearestNeighbors(n_neighbors=1).fit(images)

In [7]:
distances, indices = nbrs.kneighbors(test_imgs)
indices

array([[1324],
       [2685],
       [2357],
       ...,
       [ 535],
       [2876],
       [1645]])

In [8]:
test_labels = (np.asarray(labels).flatten()[indices.flatten()])

In [9]:
test_labels

array([129.38644905, 168.28437717, 133.05878848, ..., 352.70579474,
       188.70904405, 200.46733027])

In [10]:
final = pd.DataFrame({'filename':img_names,'xRot':test_labels})
final.sort_values('filename').to_csv('submit.csv',index=False)